<a href="https://colab.research.google.com/github/Tiabet/DACON_WebClick/blob/main/Webclick_automl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [2]:
# 동일한 결과 보장을 위해 Seed값을 고정합니다
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed를 42로 고정

In [5]:
!pip install flaml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.7/296.7 kB 5.7 MB/s eta 0:00:00


In [11]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 15.6 MB/s eta 0:00:00


In [6]:
from flaml import AutoML

데이터 로드

In [8]:
train_x =  pd.read_parquet('/content/drive/MyDrive/data/train_preprocessed.parquet')
test_x = pd.read_parquet('/content/drive/MyDrive/data/test_preprocessed.parquet')

In [9]:
tmp = pd.read_parquet('/content/drive/MyDrive/data/train.parquet')
train_y = tmp['Click']

In [16]:
type(train_y)

pandas.core.series.Series

In [17]:
if isinstance(train_y, pd.Series):
    # Convert the Series to a DataFrame
    train_y_df = train_y.to_frame(name='Click')

    # Balancing the dataset by sampling up to 5,569,860 samples per class
    train_y_balanced = train_y_df.groupby('Click').apply(lambda x: x.sample(min(len(x), 5569860)))

    # Dropping the multi-index created by groupby().apply()
    train_y_balanced = train_y_balanced.reset_index(drop=True)

    # Convert back to a Series if needed
    train_y_balanced_series = train_y_balanced['Click']

In [18]:
len(train_y_balanced_series)

11139720

In [19]:
train_y_balanced.to_parquet('drive/MyDrive/data/train_y.parquet', compression = 'gzip', engine = 'pyarrow', index = False)

automl 사용

In [13]:
# Initialize AutoML
automl = AutoML()

# Specify the settings
automl_settings = {
    "time_budget": 3600,  # Total time budget in seconds
    "metric": 'roc_auc',  # Evaluation metric
    "task": 'classification',  # Task type
    "log_file_name": 'automl.log',  # Log file
    "estimator_list": ['lgbm', 'xgboost', 'catboost', 'rf', 'extra_tree'],  # List of estimators to use
    "eval_method": "cv",  # Cross-validation method
    "n_splits": 3,  # Number of cross-validation splits
    "early_stop": 10
}

In [ ]:
# Train the model
automl.fit(X_train=train_x, y_train=train_y_balanced_series, **automl_settings)

[flaml.automl.logger: 05-24 01:19:16] {1680} INFO - task = classification
[flaml.automl.logger: 05-24 01:19:16] {1691} INFO - Evaluation method: cv
[flaml.automl.logger: 05-24 01:19:25] {1789} INFO - Minimizing error metric: 1-roc_auc
[flaml.automl.logger: 05-24 01:19:25] {1901} INFO - List of ML learners in AutoML Run: ['lgbm', 'xgboost', 'catboost', 'rf', 'extra_tree']
[flaml.automl.logger: 05-24 01:19:25] {2219} INFO - iteration 0, current learner lgbm
[flaml.automl.logger: 05-24 01:20:14] {2345} INFO - Estimated sufficient time budget=491011s. Estimated necessary time budget=1056s.
[flaml.automl.logger: 05-24 01:20:14] {2392} INFO -  at 178.3s,	estimator lgbm's best error=0.3441,	best estimator lgbm's best error=0.3441
[flaml.automl.logger: 05-24 01:20:14] {2219} INFO - iteration 1, current learner lgbm
